In [8]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import os
from pathlib import Path
from keras.applications import vgg19
from keras.preprocessing import image
from keras.models import model_from_json
import cv2

In [9]:
# Construct an relative path
# os.path.join(os.getcwd(), "training-data", ) --- To do later

In [10]:
# Construct the image path using the Path module
image_path = Path("./training-data/dataset1/known_images")

In [11]:
# Create array for the images and labels
images = []
labels = []

In [21]:
# Set a target size for the imagers
target_size = (391, 488)

In [22]:
# Load all the normal images and create labels for them
for img in image_path.glob("normal*.png"):
    # Load images from the disk
    img = image.image_utils.load_img(img, target_size=target_size)

    # Convert the image to numpy array
    image_array = image.image_utils.img_to_array(img)

    # Add the image to a list of images
    images.append(image_array)

    # For each dog image, the label should 1
    labels.append(0)

In [24]:
# Load all the opacity images and create labels for them
for img in image_path.glob("sick*.png"):
    # Load images from the disk
    img = image.image_utils.load_img(img, target_size=target_size)

    # Convert the image to numpy array
    image_array = image.image_utils.img_to_array(img)

    # Add the image to a list of images
    images.append(image_array)

    # For each dog image, the label should 1
    labels.append(1)

In [25]:
# Create a numpy array of all the images
X_train = np.array(images)

# Create a numpy array of all the labels
Y_train = np.array(labels)